# Appendix D

### D.1 Importing Libraries

In [91]:
import pandas as pd
import numpy as np
from datetime import *
import pickle as pkl

### D.2  Importing Data

In [92]:
#reading in data from other files 
tweets =pd.read_csv('clean_tweeets.csv').drop(['Unnamed: 0','Unnamed: 5','Unnamed: 6','Unnamed: 7'
                                               ,'Unnamed: 8','Unnamed: 9'],axis =1)
weather = pd.read_csv('weather_data')

In [93]:
tweets.head()

,date,tweets,CM,WR
0,3/15/2020 20:37,WR 47 CM 12,12,47
1,3/15/2020 17:55,WR 52 CM 22 🎾,22,52
2,3/15/2020 16:34,WR 48 CM 24 😀,24,48
3,3/15/2020 15:55,WR 23 CM 13,13,23
4,3/15/2020 15:23,WR 19\nCM 6 🏃🏾‍♀️😊👍🏼🌬😷,6,19


### D.3 Data Cleansing

In [94]:
#dropping rows with na values
weather_nona = weather[weather['Time'].notna()]


In [95]:
#Function rount_time 
#inputs 
#dt datetime object
#direction what way to rount_to as string up or down
#rount_to amount to rount_to to
#outputs rounded datetime object
def rount_to_time(dt, direction, rount_to):
    new_minute = (dt.minute // rount_to + (1 if direction == 'up' else 0)) * rount_to
    return dt + timedelta(minutes=new_minute - dt.minute)

In [96]:
#Formating the data into datetime format
time_format = []
for day in weather.Time.dropna():
    time_format.append( datetime.strptime(day, '%Y-%m-%d %H:%M:%S') ) 
    
    

In [97]:
#formating time data into datetime format
#I also adjusted the data for a timezone error and rounded up each timestamp to enarest 30 min
time = tweets.date
for i in range(len(time)):
    time[i] = round_time(datetime.strptime(time[i], '%m/%d/%Y %H:%M') -  timedelta(hours = 4),
                        'down',
                        30)
tweets.date  = time

C:\Users\Emiliano\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


### D.4 Data Exportation

In [98]:
#Merging the dataframes into one for easy access to all data
#Filling the missing values in the data as the weather repor came in hourly increments while the tweets come in half hour basis
# I will be using hourly weather reports and just fill the weather report with the neaerest data 
tweets.date = pd.to_datetime(tweets.date)
weather_nona.Time = pd.to_datetime(weather_nona.Time)
Data_missing = tweets.merge(weather_nona,how = 'left' , left_on = 'date',right_on = 'Time')
Data_missing.to_csv("Raw_Formated_Data.csv")